In [24]:
import os
import pandas as pd

In [33]:
# Set the path to the target directory
base_path = "C:/Users/Ally/Desktop/sheng/data/3/Data"

# Get a list of all subdirectories in "1"
subject_dirs = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

# Print each subdirectory name
for subdir in subject_dirs:
    print(subdir)
    
chunks_all = []


S1
S10
S2
S3
S4
S5
S6
S7
S8
S9


In [34]:
for j in range(len(subject_dirs)):
    subject_dirs[j]
    
    term_dirs = os.listdir(os.path.join(base_path, subject_dirs[j]))
    for k in range(len(term_dirs)):

        BVP_path = os.path.join(base_path, subject_dirs[j], term_dirs[k], "BVP.csv")
        EDA_path = os.path.join(base_path, subject_dirs[j], term_dirs[k], "EDA.csv")
        TEMP_path = os.path.join(base_path, subject_dirs[j], term_dirs[k], "TEMP.csv")

        # Read the CSV file
        df_BVP = pd.read_csv(BVP_path, header=None)
        df_EDA = pd.read_csv(EDA_path, header=None)
        df_TEMP = pd.read_csv(TEMP_path, header=None)

        '''
        if df_BVP.iloc[0,0] == df_EDA.iloc[0,0] and df_BVP.iloc[0,0] == df_TEMP.iloc[0,0]:
            print(1)
        else:
            print(0)
        '''
        
        df_BVP = df_BVP.iloc[2:].reset_index(drop=True)
        df_EDA = df_EDA.iloc[2:].reset_index(drop=True)
        df_TEMP = df_TEMP.iloc[2:].reset_index(drop=True)

        # downsample BVP to 32 hz
        df_BVP = df_BVP.iloc[::2].reset_index(drop=True)
        df_EDA = df_EDA.loc[df_EDA.index.repeat(8)].reset_index(drop=True)
        df_TEMP = df_TEMP.loc[df_TEMP.index.repeat(8)].reset_index(drop=True)

        # combine all three df into one data frame
        # Step 1: Find the minimum number of rows
        min_rows = min(len(df_BVP), len(df_EDA), len(df_TEMP))

        # Step 2: Cut each dataframe to the minimum number of rows
        df_BVP_cut = df_BVP.iloc[:min_rows].reset_index(drop=True)
        df_EDA_cut = df_EDA.iloc[:min_rows].reset_index(drop=True)
        df_TEMP_cut = df_TEMP.iloc[:min_rows].reset_index(drop=True)

        # Step 3: combine columns
        df= pd.concat([df_BVP_cut, df_EDA_cut, df_TEMP_cut], axis=1)
    
        # assign column names
        df.columns = ['BVP', 'EDA', 'TEMP']

        # set index as a column named "time"
        df = df.reset_index().rename(columns={'index': 'time'})
        df['subject'] = subject_dirs[j]

        # chunk df into chunks of 4 seconds with 1 second overlap
        # Parameters
        fs = 32              # Sampling frequency (Hz)
        chunk_duration = 4   # seconds
        overlap_duration = 1 # seconds

        chunk_size = chunk_duration * fs
        overlap_size = overlap_duration * fs
        step_size = chunk_size - overlap_size  # How many rows to move forward for each new chunk

        # Store all chunks
        chunks = []

        # Go through the DataFrame
        for start in range(0, len(df) - chunk_size + 1, step_size):
            chunk = df.iloc[start : start + chunk_size].reset_index(drop=True)
            chunks.append(chunk)

        # Now `chunks` is a list of DataFrames, each 128 rows, overlapping by 32 rows

        chunks_all.append(chunks)



C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Final\BVP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Final\EDA.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Final\TEMP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Midterm 1\BVP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Midterm 1\EDA.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Midterm 1\TEMP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Midterm 2\BVP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Midterm 2\EDA.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S1\Midterm 2\TEMP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S10\Final\BVP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S10\Final\EDA.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S10\Final\TEMP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S10\Midterm 1\BVP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S10\Midterm 1\EDA.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S10\Midterm 1\TEMP.csv
C:/Users/Ally/Desktop/sheng/data/3/Data\S10\Midterm 2\BVP.csv
C:/Users/Ally/Desktop/sheng/data/3/D

In [35]:
len(chunks_all)

30

In [55]:
for i, chunk in enumerate(chunks_all):
    pd.concat(chunk, axis = 0).reset_index(drop = True).to_csv(f'chunk_{i}.csv', index = False)

In [50]:
for i, chunk in enumerate(chunks_all):
    cnt = 0
    for j, c in enumerate(chunk):
        cnt = cnt + c.shape[0]
    print(i, cnt)

0 998144
1 476800
2 475008
3 984320
4 498560
5 554240
6 1080960
7 511232
8 592256
9 1101568
10 520960
11 436992
12 678656
13 498816
14 571264
15 650240
16 511872
17 512640
18 1020288
19 475776
20 605312
21 838272
22 528256
23 461312
24 763520
25 459904
26 424832
27 606080
28 540928
29 530304


In [40]:
357632/128

2794.0